In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from helperToolz.mirmazloumi import *
from pympler import asizeof

In [ ]:
# this might dock onto the sharpener process??

In [8]:
# set the year, month and day to estimate evapotranspiration for
year = 2019
month = 'July'
day = 1

# path to era5 raw data
era5_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'

# the LST acquisition time should determine which sharpened LST files are associated
LST_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/2019/Daily_AcqTime_maxLST_{year}_{month}.tif' # epsg 4326
# the DEM, SLOPE, ASPECT, LAT, LON will be used to sharpen some of the era5 variables (the the resolution of the DEM)
dem_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_LST_WARP.tif' # epsg 4326
slope_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_LST_WARP.tif' # epsg 4326
aspect_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_LST_WARP.tif' # epsg 4326
lat_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_LST_WARP.tif' # epsg 4326
lon_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_LST_WARP.tif' # epsg 4326

# the geopotential is needed for the sharpening as well
geopot_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif' # epsg 4326

# for the resampling to S2, we need a template --> sharpened LST
LST_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/maxLST_mvwin40_cv25_maxLST_{year}_{month}_{day:02d}.tif' # epsg:3035

# for NDVI calculation (estimating LAI and others), we use the S2 composite used for sharpening
# S2_file = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump/S2_20190705/Force_X_from_65_to_73_Y_from_39_to_47/Force_X_from_65_to_73_Y_from_39_to_47_Cube.vrt'
S2_file = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump/S2_20190705/Force_X_from_58_to_59_Y_from_33_to_34/Force_X_from_58_to_59_Y_from_33_to_34_Cube.vrt'
# find era5 file that matches the month of LST observation
valid_variables = sorted(list(set([file.split('/')[-2] for file in getFilelist(era5_path, '.grib', deep=True) \
                                   if 'total_column_water_vapour' not in file])))
# output of valid variables: ['100m_u_component_of_wind', '100m_v_component_of_wind', '2m_dewpoint_temperature',
# '2m_temperature', 'geopotential', 'surface_pressure', 'surface_solar_radiation_downward_clear_sky']

# get a list for those era5 files that match the year and month of the provided LST acquisition file
era5_path_list = find_grib_file(getFilelist(era5_path, '.grib', deep=True), LST_acq_file)
era5_path_list = [path for path in era5_path_list if any(variable in path for variable in valid_variables)] # era5 are epsg 4326 and still will be after warping to doy

In [ ]:
eraL = []
# load the era5 variable into cache at LST resolution and read-in the modelled times (one time step per band)
for path in era5_path_list:
    # check if DEM sharpener needs to be applied
    if any(var in path for var in ['100m_u_component_of_wind', '100m_v_component_of_wind', 'geopotential']):
        # do the warping without sharpening
        eraL.append(get_warped_ERA5_at_doy(path, LST_acq_file, day)) 
    elif 'downward' in path:
         eraL.append(get_ssrdsc_warped_and_corrected_at_doy(path,
                                                            LST_acq_file, day, slope_path=slope_path, aspect_path=aspect_path,
                                                            dem_path=dem_path, lat_path=lat_path,lon_path=lon_path))
    else: 
        # do warping with sharping
        # however, variables need different sharp_rates:
        # 2m_air = STANDARD --> can be excluded and loaded directly, but must be masked, probably!!
        # 2m dew = MOIST
        # surface pressure = DEM

        eraL.append(get_warped_ERA5_at_doy(path, LST_acq_file, 1, sharp_blendheight=100, sharp_DEM=dem_path, sharp_geopot=geopot_path, sharp_rate=MOIST_ADIABAT))

In [ ]:
# calculate windspeed
wind_speed = calc_wind_speed(eraL[0], eraL[1])
# delete u, v wind
eraL = eraL[2:] # removes index 0 & 1
#'2m_dewpoint_temperature','2m_temperature', 'geopotential', 'surface_pressure', 'surface_solar_radiation_downward_clear_sky'

In [ ]:
# calculate the NDVI from the S2 composite (following formula from force --> bandnames: (NIR - RED) / (NIR + RED))
S2_ds = gdal.Open(S2_file)
for idx, bname in enumerate(getBandNames(S2_file)):
    if bname == 'RED':
        red = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    elif bname == 'NIR':
        nir = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    else:
        continue
ndvi_20 = (nir - red) / (nir + red)
ndvi_20 = np.ma.masked_invalid(ndvi_20)
LAI_np = 0.57*np.exp(2.33*ndvi_20)
LAI_pos = np.ma.masked_where(LAI_np < 0, LAI_np)


In [ ]:
# bring all variables down to 20m
wind_speed_20 = warp_np_to_reference(wind_speed, geopot_path, LST_file)

In [ ]:
output = TSEB.TSEB_PT(Tr_K, vza, T_A_K, wind_speed_20, ea, p, Sn_C, Sn_S, L_dn, LAI_pos, h_C, emis_C, emis_S, 
                      z_0M, d_0, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, 
                      kB=0.0, massman_profile=None, verbose=True)

In [ ]:
MOIST_ADIABAT

In [ ]:
total_size = asizeof.asizeof(ds)  # size of wrapper object

for i in range(ds.RasterCount):
    band = ds.GetRasterBand(i + 1)
    arr = band.ReadAsArray()
    total_size += asizeof.asizeof(arr)  # include array size

print(f"Estimated memory used by in-memory raster: {total_size / 1024**2:.2f} MB")

In [ ]:
## loops over LST files

# find time-matching NDVI file (20m)
# open hourly data:
        # u_wind
        # v_wind
        # air Temp (DEM correct)
        # 2m dew point (DEM correct)              
        # surface pressure (DEM correct)
        # surfacesolarradiation (DEM correct)
        # surfacesolarradiation mean (year??) (DEM correct)
        # geopotential

# open dem_srtm 
# open VZA

# estimate LAI based on NDVI
        NDVI_data = NDVI_20m.read(1)
        NDVI2 = np.ma.masked_invalid(NDVI_data)
        LAI_np = 0.57*np.exp(2.33*NDVI2)
        LAI3 = np.ma.masked_where(LAI_np < 0, LAI_np)

# calculate windspeed and downscale
        calc_wind_speed(u, v)

# downscale surface pressure, geopotential, surfacesolarradiation + mean, dem
# donwscale airtemp + 2mdew with dem_sharpener

# calculate sun angles (zenith and azimuth as in calculate_solar_incidence_angle_flat_tilt.py)

        # estimate canopy height from estimated LAI
        hc_from_lai(lai, hc_max, lai_max, hc_min=0)
       

# estimate long wave irradiance
        ea = TSEB.met.calc_vapor_pressure(T_K=dt2_dwscaled)
        calc_longwave_irradiance(ea, t_a_k, p, z_T, h_C)
 

# calculate shortwave radiation of soil and canopy
        difvis, difnir, fvis, fnir = TSEB.rad.calc_difuse_ratio(S_dn = ssrd_dwscaled, sza = np.mean(sza))
        
        skyl = difvis * fvis + difnir * fnir
        S_dn_dir = ssrd_dwscaled * (1.0 - skyl)
        S_dn_dif = ssrd_dwscaled * skyl

        
        # Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
        rho_vis_C=np.full(LAI_pos.shape, 0.05, np.float32)
        tau_vis_C=np.full(LAI_pos.shape, 0.08, np.float32)
        rho_nir_C=np.full(LAI_pos.shape, 0.32, np.float32)
        tau_nir_C=np.full(LAI_pos.shape, 0.33, np.float32) 
        
        # Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
        rho_vis_S=np.full(LAI_pos.shape, 0.07, np.float32)
        rho_nir_S=np.full(LAI_pos.shape, 0.25, np.float32)

        # F = local LAI
        F = LAI_pos / 1
        # calculate clumping index
        Omega0 = TSEB.CI.calc_omega0_Kustas(LAI = LAI_np, f_C = 1, x_LAD=1)
        Omega = TSEB.CI.calc_omega_Kustas(Omega0, np.mean(sza))
        LAI_eff = F * Omega

        
        Sn_C, Sn_S = TSEB.rad.calc_Sn_Campbell(lai = LLAI_posAI3, sza = np.mean(sza), S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                               fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                               tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=LAI_eff)

# calculate other roughness stuff
        z_0M, d = TSEB.res.calc_roughness(LAI=np.nanmean(LAI_pos), h_C=hc, w_C=1, landcover=11, f_c=None)
        z_0M_array = np.full(LAI_pos.shape, z_0M)
        d_array = np.full(LAI_pos.shape, d)
        fg = calc_fg_gutman(ndvi = NDVI2, ndvi_min = np.nanmin(NDVI_data), ndvi_max = np.nanmax(NDVI_data))


# calculate Evapotranspiration
        Tr_K = LST2
        # float. Radiometric composite temperature (Kelvin).
        vza = VZA_dwscaled
        # float. View Zenith Angle (degrees).
        T_A_K = ta_dwscaled
        # float. Air temperature (Kelvin).
        u =  wind_speed_dwscaled  ## ***ERA5 two para
        # float. Wind speed above the canopy (m s-1).
        ea = ea
        # float. Water vapour pressure above the canopy (mb).
        p = sp_dwscaled 
        # float. Atmospheric pressure (mb), use 1013 mb by default.
        Sn_C = Sn_C
        # float. Canopy net shortwave radiation (W m-2).
        Sn_S =  Sn_S
        # float. Soil net shortwave radiation (W m-2).
        L_dn = L_dn
        # float. Downwelling longwave radiation (W m-2).
        LAI = LAI_pos
        # float. Effective Leaf Area Index (m2 m-2).
        h_C = hc 
        # float. Canopy height (m).
        emis_C = 0.98
        # float. Leaf emissivity.
        emis_S = 0.95
        # float. Soil emissivity.
        z_0M = z_0M_array 
        # float. Aerodynamic surface roughness length for momentum transfer (m).
        d_0 = d_array 
        # float. Zero-plane displacement height (m).
        z_u = 100 
        # float. Height of measurement of windspeed (m).
        z_T = 100 
        # float. Height of measurement of air temperature (m).
        # leaf_width : 
        # float, optional average/effective leaf width (m).
        z0_soil = 0.01    #optional
        # float, bare soil aerodynamic roughness length (m).
        alpha_PT = 1.26  #optional
        #  float, optional. Priestley Taylor coeffient for canopy potential transpiration, use 1.26 by default.
        x_LAD = 1        #optional
        # float. Campbell 1990 leaf inclination distribution function chi parameter.
        f_c = 1          #optional
        #float. Fractional cover.
        f_g = fg        # optional
        # float. Fraction of vegetation that is green.
        w_C = 1       #optional
        # float. Canopy width to height ratio.
        
        print('all parameters are ready for the et estimation')
        
        output = TSEB.TSEB_PT(Tr_K, vza, T_A_K, u, ea, p, Sn_C, Sn_S, L_dn, LAI_pos, h_C, emis_C, emis_S, 
                              z_0M, d_0, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, 
                              kB=0.0, massman_profile=None, verbose=True)
        
        len(output)
        ld = output[6]/ssrd_dwscaled
        heat_latent_scaled = ssrd_m_dwscaled * ld
        et_daily_p = TSEB.met.flux_2_evaporation(heat_latent_scaled, t_k=T_A_K, time_domain=24)